### Paso 1 - Leer los archivos CSV usando "DataFrameReader" de Spark

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
dbutils.widgets.text("p_environment","")
v_environment = dbutils.widgets.get("p_environment")

dbutils.widgets.text("p_file_date","2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

In [0]:
prductions_company_schema = StructType(fields=[
                    StructField("companyId", IntegerType(), True),
                    StructField("companyName", StringType(), True)])

In [0]:
productions_compnanies_df = spark.read\
    .schema(prductions_company_schema)\
    .csv(f"{bronze_folder_path}/{v_file_date}/production_company")

In [0]:
productions_compnanies_df.count()

### Paso 2 - Renombrar las columns y añadir nuevas columnas

In [0]:
from pyspark.sql.functions import col, current_timestamp, lit

productions_company_final_df = productions_compnanies_df\
    .withColumnsRenamed({"companyId":"company_id",
                         "companyName":"company_name"})\
    .withColumn("ingestion_time", current_timestamp())\
    .withColumn("enviroment", lit(v_environment))\
    .withColumn("file_date", lit(v_file_date))

In [0]:
display(productions_company_final_df)

### Paso 3- Escribir la salida en fomrato parquet

In [0]:
#overwrite_partition(productions_company_final_df, "movie_silver", "productions_companies")

In [0]:
merge_condicion = "tgt.company_id = src.company_id AND tgt.file_date = src.file_date"
merge_delta_lake(productions_company_final_df, "movie_silver", "productions_companies", silver_folder_path, merge_condicion, "file_date")

In [0]:
%sql
SELECT file_date, COUNT(1)
FROM movie_silver.productions_companies
GROUP BY file_date;

In [0]:
%sql
SELECT * FROM movie_silver.productions_companies